In [ ]:
arrow::read_parquet(here("data", "featureless.parquet"), as_data_frame = TRUE) -> atp
arrow::read_parquet(here("data", "atpTrans.parquet")) -> atpFull
library(skimr)

In [ ]:
# features q faltam:
# numero de jogos jogados - diferenca/numero de wins/winrate
# frequencia de jogos do pais de naturalidade
# se pais tem jogos - condicao/contagem/diferenca/diferenca de condicao
# tipo do torneio (afonse)
# frequencia do torneio
# premio (precisa eda) 
# distancia da ronda final (chaves)
# categoria de rank (eda)

# Categorias de torneio

Superiores:
    - Grand Slams (melhor de 5)
    - Masters 1000 (considerado o 5º Grand Slam) (melhor de 3)

Intermédios:
    - ATP 500 (terceiro nível de torneios, distribuindo 500 pontos ao campeão)
    - ATP 250 (distribui 250 pontos ao campeão)
    - ATP Finals (reúne os oito primeiros colocados do ranking ao final da temporada. Possui grande importância no calendário dos tenistas, 
                  algo próximo aos torneios Masters 1000)
    - Copa Davis (Onde ocorre a dispuita tradicional entre países)

Menores:
    - Challenger (tenistas tentam somar pontos que os permitam participar de competições mais importantes, ingressando no circuito da ATP)
    - Futures (torneios de categoria inferior ao nível Challenger)

# Número de jogos jogados

In [ ]:
atp %>%
    group_by(winner.name, winner.id) %>% 
    summarise(player.nwins = n()) %>% 
    rename(player.name = winner.name, player.id = winner.id) -> a
atp %>%
    group_by(looser.name, looser.id) %>% 
    summarise(player.nlosses = n()) %>% 
    rename(player.name = looser.name, player.id = looser.id) -> b

full_join(a, b) %>% mutate(
    player.nwins   = ifelse(player.nwins %>% is.na, 0, player.nwins),
    player.nlosses = ifelse(player.nlosses %>% is.na, 0, player.nlosses),
    player.njogos  = player.nwins + player.nlosses,
    player.winrate = player.nwins*100 / player.njogos,
) -> playerRate
playerRate

group_by: 2 grouping variables (winner.name, winner.id)

summarise: now 901 rows and 3 columns, one group variable remaining (winner.name)

rename: renamed 2 variables (player.name, player.id)

group_by: 2 grouping variables (looser.name, looser.id)

summarise: now 1,194 rows and 3 columns, one group variable remaining (looser.name)

rename: renamed 2 variables (player.name, player.id)

Joining with `by = join_by(player.name, player.id)`
full_join: added one column (player.nlosses)

           > rows only in x      17

           > rows only in y     310

           > matched rows       884

           >                 =======

           > rows total       1,211

mutate (grouped): converted 'player.nwins' from integer to double (310 fewer NA)

                  converted 'player.nlosses' from integer to double (17 fewer NA)

                  new variable 'player.njogos' (double) with 79 unique values and 0% NA

                  new variable 'player.winrate' (double) with 166 unique

player.name,player.id,player.nwins,player.nlosses,player.njogos,player.winrate
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Aaron Addison,ad06,8,37,45,17.77778
Aaron Krickstein,k023,20,6,26,76.92308
Adam Feeney,f491,39,54,93,41.93548
Adrian Garcia,g414,3,3,6,50.00000
Adrian Mannarino,me82,4,5,9,44.44444
Adrian Voinea,v166,3,2,5,60.00000
Agustin Calleri,c478,1,2,3,33.33333
Akira Santillan,sq80,16,19,35,45.71429
Albert Chang,c308,1,2,3,33.33333


In [ ]:
atp %>% 
    left_join(playerRate, by=join_by(winner.name == player.name, winner.id == player.id)) %>% 
    rename(
        winner.Anwins   = player.nwins, 
        winner.Anlosses = player.nlosses, 
        winner.Anjogos  = player.njogos, 
        winner.Awinrate = player.winrate
    ) %>% left_join(playerRate, by=join_by(looser.name == player.name, looser.id == player.id)) %>% 
    rename(
        looser.Anwins   = player.nwins, 
        looser.Anlosses = player.nlosses, 
        looser.Anjogos  = player.njogos, 
        looser.Awinrate = player.winrate
    ) %>% mutate(
        match.AnjogosDiff = winner.Anjogos - looser.Anjogos
    ) -> atpP
atpP %>% glimpse()

left_join: added 4 columns (player.nwins, player.nlosses, player.njogos, player.winrate)



ERROR: [1m[33mError[39m in `dplyr::common_by()`:[22m
[1m[22m[33m![39m `by` must be a (named) character vector, list, or NULL for natural joins (not recommended in production code),
  not a <dplyr_join_by> object.


In [ ]:
atpFull %>%
    group_by(winner.name, winner.id) %>% 
    summarise(player.nwins = n()) %>% 
    rename(player.name = winner.name, player.id = winner.id) -> a
atpFull %>%
    group_by(looser.name, looser.id) %>% 
    summarise(player.nlosses = n()) %>% 
    rename(player.name = looser.name, player.id = looser.id) -> b

full_join(a, b) %>% mutate(
    player.nwins   = ifelse(player.nwins %>% is.na, 0, player.nwins),
    player.nlosses = ifelse(player.nlosses %>% is.na, 0, player.nlosses),
    player.njogos  = player.nwins + player.nlosses,
    player.winrate = player.nwins*100 / player.njogos,
) -> playerRateF
playerRateF

In [ ]:
atpP %>% 
    left_join(playerRateF, by=join_by(winner.name == player.name, winner.id == player.id)) %>% 
    rename(
        winner.nwins   = player.nwins, 
        winner.nlosses = player.nlosses, 
        winner.njogos  = player.njogos, 
        winner.winrate = player.winrate
    ) %>% left_join(playerRate, by=join_by(looser.name == player.name, looser.id == player.id)) %>% 
    rename(
        looser.nwins   = player.nwins, 
        looser.nlosses = player.nlosses, 
        looser.njogos  = player.njogos, 
        looser.winrate = player.winrate
    ) %>% mutate(
        match.njogosDiff = winner.njogos - looser.njogos
    ) -> atpPP
atpPP %>% glimpse()

In [ ]:
atpFull %>% 
    group_by(tournament.location, tournament.name, tournament.date_start) %>%
    summarise() %>%
    group_by(tournament.location) %>%
    summarise(tournament.locationFreq = n()) -> tfreq

atpPP %>% 
    left_join(tfreq, by=join_by(winner.bornAt == tournament.location)) %>% 
    rename(winner.bornAT.tournFreq = tournament.locationFreq) %>%
    left_join(tfreq, by=join_by(looser.bornAt == tournament.location)) %>% 
    rename(looser.bornAT.tournFreq = tournament.locationFreq) %>%
    mutate(
        winner.bornAt.hasTourn = winner.bornAT.tournFreq != 0,
        looser.bornAt.hasTourn = looser.bornAT.tournFreq != 0,
        match.playersBorn.hasTournXor = winner.bornAt.hasTourn == looser.bornAt.hasTourn,
        match.playersBorn.hasTournAny = any(winner.bornAt.hasTourn, looser.bornAt.hasTourn),
        match.playersBorn.tournFreqDiff = winner.bornAt.hasTourn - looser.bornAt.hasTourn,
    )
atpPPT %>% glimpse()

In [ ]:
# atpPPT %>% SmartEDA::ExpReport(Target = "match.setsCount", op_file="temp.html")
# atpPPT %>% SmartEDA::ExpNumViz(target = "match.setsCount", col = c("Red", "Blue"), fname="temp.scatters", scatter=T)

In [ ]:
atpPPT %>% filter(winner.nwins != winner.Anwins)

Coisas a notar: 

* tournament duration é melhor ser categórico ou ordinal
* height sugere ser melhor ordinal ou range
* prize precisa de mais eda mas probs deve ser range ou até binario
* freq tourneio deve ser preciso ser categorico ou normalizar 
* diff rank deve ser normalizado tmb (posivitive skew)
* possivelmente seja bom dar oversample de domHands e tournament.ground (embora eles estejam igualmene distribuidos na var (n me lembro se é bom ou mau)) 
* match.setCount está 2/3 3 jogos
* secalhar seria bom agrupar backhands, tournament.quarter
* tournament duration está 91/8, nem sei o q fazer com isso
* as categoricas estão todas igualmente distribuidas em termos da target
* relacoes visiveis nos scatters:
    * 